In [16]:
import pybedtools
import pandas as pd
import subprocess

In [3]:
# Load the probe file
probe_input = "kmers_selected_probes.csv"
probe_df = pd.read_csv(probe_input)

In [4]:
# Rearrange and rename columns to match BED format
probe_bed_df = probe_df[["seqname", "start", "end", "gene_ID", "strand"]]
probe_bed_df.columns = ["chrom", "chromstart", "chromend", "name", "strand"]

In [5]:
probe_output = "probes_DAB1.bed"
probe_bed_df.to_csv(probe_output, sep="\t", index=False, header=False)

In [7]:
import subprocess
import pandas as pd

# Paths to your BED files
probe_bed = "probes_DAB1.bed"  # Your probes file
canonical_bed = "canonical_transcripts_cleaned.bed"  # File containing canonical transcript intervals
output_bed = "annotated_probes.bed"  # Output file to store annotated results

# BEDTools intersect command
cmd = [
    "bedtools", "intersect",
    "-a", probe_bed,
    "-b", canonical_bed,
    "-wao"
]

# Run BEDTools
with open(output_bed, "w") as output:
    subprocess.run(cmd, stdout=output)

# Load the results for analysis
df = pd.read_csv(output_bed, sep="\t", header=None)

# Assign column names including the strand
df.columns = [
    "probe_chr", "probe_start", "probe_end", "probe_info", "probe_strand",
    "canonical_chr", "canonical_start", "canonical_end", "canonical_info", "canonical_strand",
    "overlap"
]

# Quantify coverage
coverage_summary = df.groupby("probe_info")["overlap"].sum()

# Display annotated probes with strand information
print(df.head())
print("\nCoverage summary by probe:")
print(coverage_summary)

  probe_chr  probe_start  probe_end probe_info probe_strand canonical_chr  \
0      chr1     57291007   57291127       DAB1            -          chr1   
1      chr1     57015182   57015302       DAB1            -          chr1   
2      chr1     57306924   57307044       DAB1            -          chr1   
3      chr1     56997957   56998077       DAB1            -          chr1   
4      chr1     56995847   56995967       DAB1            -          chr1   

   canonical_start  canonical_end canonical_info canonical_strand  overlap  
0         56994778       57424060           DAB1                -      120  
1         56994778       57424060           DAB1                -      120  
2         56994778       57424060           DAB1                -      120  
3         56994778       57424060           DAB1                -      120  
4         56994778       57424060           DAB1                -      120  

Coverage summary by probe:
probe_info
DAB1    1320
Name: overlap, dtype: i

In [5]:
#!bedtools intersect -a canonical_transcripts_cleaned.bed -b probes_DAB1.bed -wao > intersect_output.bed


In [8]:
!conda install -c bioconda gffread -y


Channels:
 - bioconda
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /gpfs/commons/home/cpizzano/miniconda3

  added / updated specs:
    - gffread


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2024.12.14         |  py312h06a4308_0         161 KB
    conda-24.11.3              |  py312h06a4308_0         1.1 MB
    gffread-0.12.7             |       hdcf5f25_4         131 KB  bioconda
    ------------------------------------------------------------
                                           Total:         1.4 MB

The following NEW packages will be INSTALLED:

  gffread            bioconda/linux-64::gffread-0.12.7-hdcf5f25_4 

The following packages will be UPDATED:

  conda              conda-forge::conda-24.11.2-py312h7900~ --> pkgs/main::conda-24.11.3-py312h06a4308_0 

The following packages will be SUPERS

Exon BED file created: exons.bed


In [21]:
import pandas as pd

# Path to your GTF file
gtf_file = "gencode.v47.annotation.gtf"
gene_of_interest = "DAB1"  # Replace with your gene of interest

# Create an empty list to store exon data
exons = []

# Read the GTF file line by line
with open(gtf_file, "r") as f:
    for line in f:
        # Skip comment or header lines
        if line.startswith("#"):
            continue
        # Split the line into tab-separated fields
        fields = line.strip().split("\t")
        # Check if the feature type is 'exon' and if the gene matches the one of interest
        if fields[2] == "exon" and f'gene_name "{gene_of_interest}"' in fields[8]:
            chrom = fields[0]  # Chromosome
            start = int(fields[3]) - 1  # Convert to 0-based BED format (subtract 1 from start)
            end = int(fields[4])  # End coordinate
            info = fields[8]  # Attribute field
            strand = fields[6]  # Strand information
            exons.append([chrom, start, end, info, ".", strand])  # Add to list

# Check if any exons were found
if len(exons) == 0:
    print(f"No exons found for gene {gene_of_interest}. Please check your GTF file.")
else:
    # Convert the list to a DataFrame
    exons_df = pd.DataFrame(exons, columns=["chrom", "start", "end", "info", "score", "strand"])
    
    # Save to a BED file
    output_bed = f"{gene_of_interest}_exons.bed"
    exons_df.to_csv(output_bed, sep="\t", header=False, index=False)
    print(f"Exon BED file created: {output_bed}")



Exon BED file created: DAB1_exons.bed


In [23]:
!bedtools intersect -a DAB1_exons.bed -b probes_DAB1.bed -wao > intersect_output.bed


In [24]:
import pandas as pd

# Load the intersect output
intersect_df = pd.read_csv("intersect_output.bed", sep="\t", header=None)
intersect_df.columns = [
    "exon_chr", "exon_start", "exon_end", "exon_info", "exon_score", "exon_strand",
    "probe_chr", "probe_start", "probe_end", "probe_info", "probe_score", "overlap"
]

# Calculate exon lengths
intersect_df["exon_length"] = intersect_df["exon_end"] - intersect_df["exon_start"]

# Sum overlap by exon
exon_coverage = intersect_df.groupby("exon_info")["overlap"].sum()

# Merge with exon lengths
exon_summary = pd.DataFrame({
    "total_covered_bases": exon_coverage,
    "exon_length": intersect_df.groupby("exon_info")["exon_length"].first()
})
exon_summary["coverage_fraction"] = exon_summary["total_covered_bases"] / exon_summary["exon_length"]

print(exon_summary)


                                                    total_covered_bases  \
exon_info                                                                 
gene_id "ENSG00000173406.16"; transcript_id "EN...                   58   
gene_id "ENSG00000173406.16"; transcript_id "EN...                    0   
gene_id "ENSG00000173406.16"; transcript_id "EN...                    0   
gene_id "ENSG00000173406.16"; transcript_id "EN...                    0   
gene_id "ENSG00000173406.16"; transcript_id "EN...                   24   
...                                                                 ...   
gene_id "ENSG00000173406.16"; transcript_id "EN...                  120   
gene_id "ENSG00000173406.16"; transcript_id "EN...                    0   
gene_id "ENSG00000173406.16"; transcript_id "EN...                    0   
gene_id "ENSG00000173406.16"; transcript_id "EN...                    0   
gene_id "ENSG00000173406.16"; transcript_id "EN...                  120   

                        

In [25]:
!bedtools subtract -a DAB1_exons.bed -b probes_DAB1.bed > uncovered_exons.bed


In [26]:
uncovered_df = pd.read_csv("uncovered_exons.bed", sep="\t", header=None)
uncovered_df.columns = ["chrom", "start", "end", "info", "score", "strand"]
print(uncovered_df)


    chrom     start       end  \
0    chr1  57423929  57424060   
1    chr1  57290963  57291007   
2    chr1  57291127  57291166   
3    chr1  57145289  57145429   
4    chr1  57136542  57136641   
..    ...       ...       ...   
137  chr1  57290963  57291007   
138  chr1  57291127  57291166   
139  chr1  57145416  57145429   
140  chr1  57883998  57884016   
141  chr1  57826070  57826455   

                                                  info score strand  
0    gene_id "ENSG00000173406.16"; transcript_id "E...     .      -  
1    gene_id "ENSG00000173406.16"; transcript_id "E...     .      -  
2    gene_id "ENSG00000173406.16"; transcript_id "E...     .      -  
3    gene_id "ENSG00000173406.16"; transcript_id "E...     .      -  
4    gene_id "ENSG00000173406.16"; transcript_id "E...     .      -  
..                                                 ...   ...    ...  
137  gene_id "ENSG00000173406.16"; transcript_id "E...     .      -  
138  gene_id "ENSG00000173406.16"; transcri